In [31]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import random
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model # type: ignore

print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

def seed_tensorflow(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-10-23 01:25:42.469868: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 01:25:42.470019: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 01:25:42.470081: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [32]:
# binary_train_label
# short_play
# effective_play
# long_play
# complete_play

# base_model = "./model/kuairand_backbone/mlp_{}.h5".format(select_label)
CPF_model = "./model/kuairand_CPF/mlp_{}.h5".format('CPF')

In [33]:
para = {'data_path':'./kuai_input/',
        'embedding_dim':64,
        'seed':0,
        'lr':1e-5,
        'batch_size':512,
        'epochs':64,
        'verbose':1,
        'callback':{
                     'monitor':'val_auc',
                     'patience':10,
                     'CPF_model':CPF_model
                     },
        'mlp_dims':[256,128,64],
        'mlp_act':'relu',
        'mlp_dps':[.5,.5,.5],
       }


In [34]:
action = pd.read_csv(para['data_path']+"action.csv")
feed_emb = np.load(para['data_path']+"embeddings.npy")
action = action[['user_id','video_id','duration_level','tab','user_active_degree','author_id','music_id',
                 'short_play', 'binary_train_label', 'effective_play',
                 'long_play', 'complete_play','date']]
categories_to_keep = ['full_active', 'high_active', 'middle_active','low_active']

action = action[action['user_active_degree'].isin(categories_to_keep)]

category_mapping = {
    'low_active': 0,
    'middle_active': 1,
    'high_active': 2,
    'full_active': 3
}

action['user_active_degree_encoded'] = action['user_active_degree'].map(category_mapping)
action['music_id_encoded'], _ = pd.factorize(action['music_id'])

In [35]:
#action = action.sort_values(by=['date'])
train= action.groupby('user_id').apply(lambda x: x[:int(len(x)*0.6)]).reset_index(drop=True)
valid= action.groupby('user_id').apply(lambda x: x[int(len(x)*0.6):int(len(x)*0.8)]).reset_index(drop=True)
test= action.groupby('user_id').apply(lambda x: x[int(len(x)*0.8):]).reset_index(drop=True)

In [81]:
seed_tensorflow(seed=para['seed'])
"""
    uid_lay = get_layer((1,),'uid',d1 = max(action['user_id'])+1,d2 = para['embedding_dim'],trainable=True)
    videoid_lay = get_layer((1,),'video_id',d1 = max(action['video_id'])+1,d2 = int(para['embedding_dim']),trainable=True)
    duration_lay = get_layer((1,),'duration_id',d1 = max(action['duration_level'])+1,d2 = int(para['embedding_dim']),trainable=True)
    tab_lay = get_layer((1,),'tab',d1 = max(action['tab'])+1,d2 = int(para['embedding_dim']),trainable=True)
    user_active_degree_encoded = get_layer((1,),'user_active_degree_encoded',d1 = max(action['user_active_degree_encoded'])+1,d2 = int(para['embedding_dim']),trainable=True)
    author_lay = get_layer((1,),'author_id',d1 = max(action['author_id'])+1,d2 = int(para['embedding_dim']),trainable=True)
    music_lay = get_layer((1,),'music_id_encoded',d1 = max(action['music_id_encoded'])+1,d2 = int(para['embedding_dim']),trainable=True)
"""

features = ['user_id','video_id','duration_level','tab','user_active_degree_encoded','author_id','music_id_encoded','video_id','short_play','binary_train_label','effective_play'
            ,'long_play','complete_play']

# # short_play binary_train_label effective_play long_play complete_play
def get_input(df,is_test=False):
    X = []
    for f in features:
        X.append(df[f].values.reshape(-1,1))
    labels = ['short_play', 'binary_train_label', 'effective_play', 'long_play', 'complete_play']
    
    # Create a list of label arrays, one for each task
    y = [df[label].values.reshape(-1, 1) for label in labels]
    return X,y

X_train,y_train = get_input(train,is_test=False)
X_valid,y_valid = get_input(valid,is_test=False)
X_test,y_test = get_input(test,is_test=True)

2024-10-23 02:45:44.886642: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 02:45:44.887005: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 02:45:44.887095: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [87]:
y_test[4]

array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [38]:
seed_tensorflow(seed=para['seed'])

def get_layer(shape,name,dtype='int32',d1=None,d2=None,pretrain=None,trainable=False):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=shape, name=name+'_input', dtype=dtype))
    if d1 is None:
        d1 = pretrain.shape[0]
        d2 = pretrain.shape[1]
    model.add(tf.keras.layers.Embedding(input_dim=int(d1),
                       output_dim=int(d2),
                       weights=[pretrain] if pretrain is not None else None,
                       trainable=trainable,
                       name=name+'_embedding'))
    return model
#'user_id','video_id','duration_level','tab','user_active_degree','author_id','music_id'
def mlp_bias(TRAIN):
    uid_lay = get_layer((1,),'uid',d1 = max(action['user_id'])+1,d2 = para['embedding_dim'],trainable=TRAIN)
    videoid_lay = get_layer((1,),'video_id',d1 = max(action['video_id'])+1,d2 = int(para['embedding_dim']),trainable=TRAIN)
    duration_lay = get_layer((1,),'duration_id',d1 = max(action['duration_level'])+1,d2 = int(para['embedding_dim']),trainable=TRAIN)
    tab_lay = get_layer((1,),'tab',d1 = max(action['tab'])+1,d2 = int(para['embedding_dim']),trainable=TRAIN)
    user_active_degree_encoded = get_layer((1,),'user_active_degree_encoded',d1 = max(action['user_active_degree_encoded'])+1,d2 = int(para['embedding_dim']),trainable=TRAIN)
    author_lay = get_layer((1,),'author_id',d1 = max(action['author_id'])+1,d2 = int(para['embedding_dim']),trainable=TRAIN)
    music_lay = get_layer((1,),'music_id_encoded',d1 = max(action['music_id_encoded'])+1,d2 = int(para['embedding_dim']),trainable=TRAIN)
    
    pre_lay = get_layer((1,),'pretrain',pretrain = feed_emb,trainable=False)
    label_input_SP = tf.keras.layers.Input(shape=(1,), name='SP')
    label_input_BP = tf.keras.layers.Input(shape=(1,), name='BP')
    label_input_EP = tf.keras.layers.Input(shape=(1,), name='EP')
    label_input_LP = tf.keras.layers.Input(shape=(1,), name='LP')
    label_input_CP = tf.keras.layers.Input(shape=(1,), name='CP')

    Lay_bi =[uid_lay,videoid_lay,duration_lay,tab_lay,user_active_degree_encoded,author_lay,music_lay,pre_lay]


    lay_bi_outs = []
    for l in Lay_bi:
        lay_bi_outs += l.outputs

    vec = tf.keras.layers.concatenate(lay_bi_outs,axis=-1)
    vec = tf.squeeze(vec,axis=1)
    
    #vec = tf.keras.layers.BatchNormalization()(vec)

    vec = tf.keras.layers.Dense(256,
                                    activation = 'relu',
                                    name='mlp_dense0')(vec)

    vec = tf.keras.layers.Dropout(0.5)(vec)

    vec = tf.keras.layers.Dense(128,
                                    activation = 'relu',
                                    name='mlp_dense1')(vec)

    vec = tf.keras.layers.Dropout(0.5)(vec)

    instance = tf.keras.layers.Dense(64,
                                    activation = 'relu',
                                    name='instance')(vec)
    #instance = tf.keras.layers.Dropout(0.5)(vec)

    #instance = tf.keras.layers.BatchNormalization(name='instance')(vec)

    model_inputs = []
    for l in Lay_bi:
        model_inputs += l.inputs
    model_inputs += [label_input_SP]
    model_inputs += [label_input_BP]
    model_inputs += [label_input_EP]
    model_inputs += [label_input_LP]
    model_inputs += [label_input_CP]
    model = tf.keras.Model(inputs=model_inputs,outputs=[instance])

    return model # 返回一个模型


2024-10-23 01:26:01.216907: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 01:26:01.217084: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 01:26:01.217150: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [39]:
from tensorflow.keras.models import Model
seed_tensorflow(seed=para['seed'])
base_model_backbone = mlp_bias(True)


# n*64
vec = base_model_backbone.outputs[0] # instance     dim=64
final_output = tf.keras.layers.Dense(1,"sigmoid")(vec)

# Create a model that includes the routed output
model = Model(inputs=base_model_backbone.inputs, outputs=final_output)

model.summary()


2024-10-23 01:26:01.223133: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 01:26:01.223297: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 01:26:01.223360: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 uid_input (InputLayer)      [(None, 1)]                  0         []                            
                                                                                                  
 video_id_input (InputLayer  [(None, 1)]                  0         []                            
 )                                                                                                
                                                                                                  
 duration_id_input (InputLa  [(None, 1)]                  0         []                            
 yer)                                                                                             
                                                                                           

# Generate Prototypes

In [40]:
proto_tensors = [tf.random.normal([10, 64], mean=0.0, stddev=1.0, dtype=tf.float32) for _ in range(5)]


# prototype generation

In [69]:
class Prototypes(tf.keras.layers.Layer):
    def __init__(self,
                 k ,
                 beta1=0.0,
                 beta2=0.0,
                 beta3=0.0,
                 init_prototypes = None,
                 **kwargs):
        super(Prototypes, self).__init__(**kwargs)
        self.k = k
        self.beta1, self.beta2, self.beta3 = beta1, beta2, beta3
        # y_train[0]
        self.init_prototypes = init_prototypes
        self.binary_classifiers = []
        # 
        for _ in range(5):
            mlp = tf.keras.Sequential([
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])
            mlp.build((None, 64))
            self.binary_classifiers.append(mlp)
            
    # define prototype
    def build(self, input_shape):
        self.prototypes = self.add_weight(
            name='prototypes',
            shape=(1, self.k, 64),
            initializer=tf.keras.initializers.Constant(self.init_prototypes[tf.newaxis, :, :]),
            trainable=True
        )
        super(Prototypes, self).build(input_shape)

    
    def orthogonality_loss(self,prototypes, k):
        # 确保传入的 k 是偶数
        assert k % 2 == 0, "k must be even."
        D = k // 2

        prototypes = tf.reshape(prototypes, [k, 64])
        prototypes = tf.nn.l2_normalize(prototypes, axis=1)

        cosine_similarity_matrix = tf.matmul(prototypes, prototypes, transpose_b=True)

        upper_triangular_part = tf.linalg.band_part(cosine_similarity_matrix, 0, -1)
        upper_triangular_part -= tf.linalg.band_part(cosine_similarity_matrix, 0, 0)  # 去掉对角线

        loss = tf.reduce_sum(tf.square(upper_triangular_part))
        normalization_factor = D * (2 * D - 1)
        loss /= normalization_factor

        return loss

    def contrastive_loss(self, vec, labels, duration):
        # Normalize the vectors
        vec_norm = tf.nn.l2_normalize(vec, axis=1)
        #print("vec_norm",vec_norm)
        sim_matrix = tf.matmul(vec_norm, vec_norm, transpose_b=True)
        #print("sim_matrix",sim_matrix)
        # Create masks
        label_eq = tf.equal(labels, tf.transpose(labels))
        #print("label_eq",label_eq)
        duration_eq = tf.equal(duration, tf.transpose(duration))
        #print("duration_eq",duration_eq)
        mask = tf.logical_and(label_eq, duration_eq)
        mask = tf.cast(mask, dtype=tf.float32)
        #print("mask",mask)

        # Compute the softmax denominator
        exp_sim = tf.exp(sim_matrix)
        total_sum = tf.reduce_sum(exp_sim)
        #print("total_sum",total_sum)
        normal_sim = exp_sim / total_sum
        #print("normal_sim",normal_sim)

        log_normal_sim = tf.math.log(normal_sim)
        weighted_log_prob = mask * log_normal_sim

        num_ones = tf.reduce_sum(mask)

        loss = -tf.reduce_sum(weighted_log_prob)
        return loss/num_ones

    def assign_loss(self,output, vec):


        output_norm = tf.nn.l2_normalize(output, axis=1)
        vec_norm = tf.nn.l2_normalize(vec, axis=1)

        cosine_similarity = tf.reduce_sum(tf.multiply(output_norm, vec_norm), axis=1)
        loss = 1 - tf.reduce_mean(cosine_similarity)
        return loss
    def get_alpha(self,n_batch,duration_one_hot_expanded,alpha_product):
        """
        alpha_product : (n,10,1)
        duration_one_hot_expanded: (n,5,1)
        """
        expanded_alpha = tf.reshape(alpha_product, [n_batch, 5, 2, 1])
        masked = expanded_alpha * tf.cast(duration_one_hot_expanded, tf.float32)[:, :, :, tf.newaxis]
        # n,2,1
        alpha_para = tf.reduce_sum(masked, axis=1)
        alpha_neg = alpha_para[:, 0, :]  # 第一个元素，保留最后一个轴
        alpha_pos = alpha_para[:, 1, :]  # 第二个元素，保留最后一个轴
        return alpha_neg,alpha_pos
    def mse_loss(self,vec,label):
        vec = tf.convert_to_tensor(vec, dtype=tf.float32)

        label = tf.convert_to_tensor(label, dtype=tf.float32)

        squeezed_label = tf.squeeze(label)

        #tf.print(label, summarize=-1)
        positive_indices = tf.where(squeezed_label == 1)
        negative_indices = tf.where(squeezed_label == 0)

        positive_samples = tf.gather(vec, positive_indices)
        negative_samples = tf.gather(vec, negative_indices)
        positive_samples = tf.squeeze(positive_samples,axis=1)
        negative_samples = tf.squeeze(negative_samples,axis=1)
        positive_loss = tf.reduce_mean(tf.square(positive_samples - self.pos_mean_prototype))
        negative_loss = tf.reduce_mean(tf.square(negative_samples - self.neg_mean_prototype))

        total_loss = positive_loss * 0.7 + negative_loss * 0.3

        return total_loss

    def binary_loss(self,alpha, labels):
        bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
        loss = bce(labels, alpha)
        return loss     
    
    def create_mask(self,duration, batch_size, num_classes=40, bucket_size=8):
        start_indices = duration * bucket_size
        masks = []
        for i in range(batch_size):
            mask = tf.concat([
                tf.zeros((start_indices[i, 0], 1), dtype=tf.float32),
                tf.ones((bucket_size, 1), dtype=tf.float32),
                tf.zeros((num_classes - start_indices[i, 0] - bucket_size, 1), dtype=tf.float32)
            ], axis=0)
            masks.append(mask)
        return tf.stack(masks)
    def binary_crossentropy_manual(self, y_true, y_pred,para):
        y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

        # 防止 log(0) 的情况发生
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        loss = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)

        weights = tf.where(y_true == 1, para, 1.0)
        weighted_loss = loss * weights

        positive_loss = tf.boolean_mask(weighted_loss, y_true == 1)
        negative_loss = tf.boolean_mask(weighted_loss, y_true == 0)
        #total_positive_loss = tf.reduce_sum(positive_loss)
        #total_negative_loss = tf.reduce_sum(negative_loss)

        return tf.reduce_mean(weighted_loss)
    def prototype_loss(self,prototypes_output):
        bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
        labels = tf.constant([[0], [1], [0], [1], [0], [1], [0], [1], [0], [1]], dtype=tf.float32)
        loss = bce(labels, prototypes_output)
        return loss
    
    def call(self, x, training=None):

        vec,duration,label = x
        a = tf.expand_dims(vec, -2)

        b = self.prototypes
        n_batch = tf.shape(vec)[0]

        dot_product = tf.multiply(a, b)

        norm_a = tf.norm(a, axis=-1, keepdims=True)  # 保持维度，使得维度是 [batch_size, 1, 1]

        norm_b = tf.norm(b, axis=-1, keepdims=True)  # 保持维度，使得维度是 [1, k, 1]

        # (batch_size, k,1)
        sum_product = tf.reduce_sum(dot_product, axis=-1, keepdims=True)
        cos_similarity = sum_product / (norm_a * norm_b)
        
        #tf.print(cos_similarity.shape)
        """
        tf.print(vec,summarize=-1)
        tf.print(self.prototypes,summarize=-1)
        tf.print("Input duration")
        tf.print(duration.shape)
        tf.print("Input label")
        tf.print(label.shape)
        #tf.print(vec)
        tf.print(duration, summarize=-1)
        tf.print(label, summarize=-1)

        tf.print(dot_product.shape)
        tf.print(dot_product)

        tf.print(sum_product.shape)
        tf.print(sum_product,summarize=-1)

        tf.print(norm_a.shape)
        tf.print(norm_a)

        tf.print(norm_b.shape)
        tf.print(norm_b)

        tf.print(cos_similarity, summarize=-1)
        """
        # （batch_size, k//2, 2, 1)
        reshaped_sum_product = tf.reshape(cos_similarity, (n_batch, self.k // 2, 2))

        temperature = 0.05

        scaled_logits = reshaped_sum_product / temperature

        softmaxed_sum_product = tf.nn.softmax(scaled_logits, axis=2)

        # (batch_size, k,1)
        alpha_product = tf.reshape(softmaxed_sum_product, (n_batch, self.k, 1))
        #tf.print(alpha_product,summarize=-1)
        """
        tf.print(alpha_product.shape)
        tf.print(alpha_product)
        """
        #self.new = alpha_product
        # n_batch, self.k, 64
        # b:(1, k, 64)
        alpha_product_expanded = tf.broadcast_to(alpha_product, [n_batch, self.k, 64])

        # calculate
        weighted_products = alpha_product_expanded * b

        # add up
        reshaped_weighted_products = tf.reshape(weighted_products, [n_batch, self.k//2, 2, 64])

        # (batchsize,5,64)
        pre_output = tf.reduce_sum(reshaped_weighted_products, axis=2)  # 对第三个维度求和

        # 1 -> [0,1,0,0,0]
        duration_one_hot = tf.one_hot(duration, depth=5)
        duration_one_hot_expanded = tf.expand_dims(duration_one_hot, -1)
        #(batch_size, 5, 1)
        duration_one_hot_expanded = tf.squeeze(duration_one_hot_expanded, axis=[1])

        masked_output = pre_output * duration_one_hot_expanded
        # output (batchsize,64)
        output = tf.reduce_sum(masked_output, axis=1)


        # get_alpha
        alpha_neg, alpha_pos = self.get_alpha(n_batch,duration_one_hot_expanded,alpha_product)
        #tf.print(alpha_pos,summarize = -1)
        #tf.print(alpha_neg,summarize = -1)
        #tf.print(alpha_pos,summarize   =-1)
        #tf.print(label, summarize=-1)
        classifier_outputs = []


        
        for i in range(5):
            classifier_output = self.binary_classifiers[i](tf.squeeze(self.prototypes,axis=0))
             # (batch_size, 1, 1)#
            #classifier_output += i
            classifier_outputs.append(classifier_output)
        
        #tf.print(classifier_outputs,summarize = -1)
        #tf.print(classifier_outputs.shape,summarize = -1)
        indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]  
        prototype_output_elements = []

        for i, idx in enumerate(indices):
            tensor_index = i // 2  
            element = tf.gather(classifier_outputs[tensor_index], idx, axis=0) 
            prototype_output_elements.append(element)

        prototype_output = tf.concat(prototype_output_elements, axis=0)
        prototype_output = tf.expand_dims(prototype_output,axis=1)
        #(1,10)
  
        positive_indices = 2 * duration + 1
        negative_indices = 2 * duration

        positive_indices = tf.clip_by_value(positive_indices, 0, prototype_output.shape[0] - 1)
        negative_indices = tf.clip_by_value(negative_indices, 0, prototype_output.shape[0] - 1)
        #tf.print(alpha_pos,summarize = -1)
        # 提取对应的 prototype_output 值
        positive_values = tf.gather(prototype_output, positive_indices)
        negative_values = tf.gather(prototype_output, negative_indices)
        #positive_values = tf.squeeze(positive_values,axis=0)
        #negative_values = tf.squeeze(negative_values,axis=0)
        output = alpha_pos * tf.squeeze(positive_values,axis=-1) 
        assign_loss = self.assign_loss(output,vec)

        binary_loss = self.binary_crossentropy_manual(label,output,1.0)
        self.add_loss(1.0 * binary_loss)
        # 5
        assign = self.binary_crossentropy_manual(label,alpha_pos,1.0)
        self.add_loss(0.05 * assign)

        ortho_loss = self.orthogonality_loss(self.prototypes,self.k)
        self.add_loss(0.05 * ortho_loss)

        contrastive_loss = self.contrastive_loss(vec,label,duration)
        self.add_loss(0.05 * contrastive_loss)

        proto_loss = self.prototype_loss(prototype_output)
        self.add_loss(0.1 * proto_loss)


        return output

In [118]:
class MLP(tf.keras.layers.Layer):
    def __init__(self,
                 **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')
    def binary_crossentropy_manual(self, y_true, y_pred,para):
        y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        loss = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
        weights = tf.where(y_true == 1, para, 1.0)
        weighted_loss = loss * weights
        return tf.reduce_mean(weighted_loss)
    def call(self, x, training=None):

        vec,duration,label = x
        output = self.output_layer(vec)
        binary_loss = self.binary_crossentropy_manual(label,output,1.0)
        self.add_loss(1.0 * binary_loss)
        return output

In [119]:
from tensorflow.keras.models import Model
seed_tensorflow(seed=para['seed'])
backbone = mlp_bias(True)
#backbone.load_weights("./model/kuairand_CPF/mlp_effect_backbone.h5")

duration_id_input_output = backbone.get_layer("duration_id_input").output
output_names = ['SP', 'BP', 'EP', 'LP', 'CP']

labels_outputs = [backbone.get_layer(name).output for name in output_names]
flag = 1
"""
Multi-task
"""
# MLP+CPF
if flag:
    outputs = []  # List to hold outputs from each Prototypes layer

    # Loop to create multiple Prototypes layers
    for i in range(5):
        prototypes_layer = Prototypes(k=10, init_prototypes=proto_tensors[i])

        # Get the output from each Prototypes layer
        output = prototypes_layer([backbone.outputs[0], duration_id_input_output, labels_outputs[i]])
        outputs.append(output)
else:
    outputs = []  # List to hold outputs from each Prototypes layer

    # Loop to create multiple Prototypes layers
    for i in range(5):
        mlp_layer = MLP()

        # Get the output from each Prototypes layer
        output = mlp_layer([backbone.outputs[0], duration_id_input_output, labels_outputs[i]])
        outputs.append(output)


model = tf.keras.Model(inputs=backbone.inputs, outputs=outputs)

seed_tensorflow(seed=para['seed'])

adam=tf.keras.optimizers.Adam(learning_rate=para['lr'])
adam = tf.keras.optimizers.Adam(learning_rate=para['lr'])
model.compile(optimizer=adam,
              metrics=['AUC'])




2024-10-23 03:04:58.441754: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 03:04:58.442065: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 03:04:58.442207: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [71]:
para['callback']['CPF_model']

'./model/kuairand_CPF/mlp_CPF.h5'

In [72]:
seed_tensorflow(seed=para['seed'])

es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_output_5_loss',  
    mode='min',          
    patience=para['callback']['patience'] 
)
checkpoint_auc = tf.keras.callbacks.ModelCheckpoint(
    filepath=para['callback']['CPF_model'],  # 保存模型的路径
    monitor='val_output_5_auc',  # 监控验证集 AUC
    mode='max',          # 保存验证 AUC 最大的模型
    save_weights_only=True,  # 只保存模型权重
    save_best_only=True  # 只保存最佳模型
)
hist = model.fit(X_train,
                 y_train,
                 epochs=50,
                 batch_size=512,
                 shuffle=True,
                 verbose=para['verbose'],
                 callbacks=[checkpoint_auc,es_callback],
                 validation_data=(X_valid,y_valid))


Epoch 1/50


2024-10-23 02:19:36.058715: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 02:19:36.059297: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 02:19:36.059641: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

1431/1431 [==============================] - 41s 24ms/step - loss: 12.4389 - prototypes_61_auc: 0.5644 - prototypes_62_auc_1: 0.5746 - prototypes_63_auc_2: 0.5528 - prototypes_64_auc_3: 0.5760 - prototypes_65_auc_4: 0.4495 - val_loss: 11.3464 - val_prototypes_61_auc: 0.6066 - val_prototypes_62_auc_1: 0.6107 - val_prototypes_63_auc_2: 0.5843 - val_prototypes_64_auc_3: 0.6379 - val_prototypes_65_auc_4: 0.6060
Epoch 2/50
1431/1431 [==============================] - 24s 17ms/step - loss: 11.4522 - prototypes_61_auc: 0.5970 - prototypes_62_auc_1: 0.5888 - prototypes_63_auc_2: 0.5867 - prototypes_64_auc_3: 0.5918 - prototypes_65_auc_4: 0.5747 - val_loss: 10.9717 - val_prototypes_61_auc: 0.6106 - val_prototypes_62_auc_1: 0.6223 - val_prototypes_63_auc_2: 0.6409 - val_prototypes_64_auc_3: 0.6613 - val_prototypes_65_auc_4: 0.6887
Epoch 3/50
1431/1431 [==============================] - 25s 17ms/step - loss: 11.1378 - prototypes_61_auc: 0.6139 - prototypes_62_auc_1: 0.5913 - prototypes_63_auc_2: 

# Test


In [104]:
seed_tensorflow(seed=para['seed'])
model.load_weights(para['callback']['CPF_model'])
y_test_pred = model.predict(X_test,batch_size=10000)

2024-10-23 02:54:58.356403: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 02:54:58.357128: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-23 02:54:58.357400: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

26/26 [==============================] - 0s 4ms/step


In [109]:
print("SP")
auc_metric = tf.keras.metrics.AUC()
auc_metric.update_state(y_test[0], y_test_pred[0])
auc = auc_metric.result().numpy()

logloss_metric = tf.keras.metrics.BinaryCrossentropy()

logloss_metric.update_state(y_test[0], y_test_pred[0])
log_loss = logloss_metric.result().numpy()
print('auc:',auc)
print('logloss:',log_loss)

SP
auc: 0.78749484
logloss: 0.5131945


In [110]:
print("BP")
auc_metric = tf.keras.metrics.AUC()
auc_metric.update_state(y_test[1], y_test_pred[1])
auc = auc_metric.result().numpy()

logloss_metric = tf.keras.metrics.BinaryCrossentropy()

logloss_metric.update_state(y_test[1], y_test_pred[1])
log_loss = logloss_metric.result().numpy()
print('auc:',auc)
print('logloss:',log_loss)

BP
auc: 0.77822256
logloss: 0.4901078


In [111]:
print("EP")
auc_metric = tf.keras.metrics.AUC()
auc_metric.update_state(y_test[2], y_test_pred[2])
auc = auc_metric.result().numpy()

logloss_metric = tf.keras.metrics.BinaryCrossentropy()

logloss_metric.update_state(y_test[2], y_test_pred[2])
log_loss = logloss_metric.result().numpy()
print('auc:',auc)
print('logloss:',log_loss)

EP
auc: 0.79135776
logloss: 0.468202


In [112]:
print("LP")
auc_metric = tf.keras.metrics.AUC()
auc_metric.update_state(y_test[3], y_test_pred[3])
auc = auc_metric.result().numpy()

logloss_metric = tf.keras.metrics.BinaryCrossentropy()

logloss_metric.update_state(y_test[3], y_test_pred[3])
log_loss = logloss_metric.result().numpy()
print('auc:',auc)
print('logloss:',log_loss)

LP
auc: 0.7979415
logloss: 0.38339123


In [123]:
print("CP")
auc_metric = tf.keras.metrics.AUC()
auc_metric.update_state(y_test[4], y_test_pred[4])
auc = auc_metric.result().numpy()

logloss_metric = tf.keras.metrics.BinaryCrossentropy()

logloss_metric.update_state(y_test[4], y_test_pred[4])
log_loss = logloss_metric.result().numpy()
print('auc:',auc)
print('logloss:',log_loss)

CP
auc: 0.81331967
logloss: 0.32460374
